# 训练深层神经网络


## 梯度消失/爆炸问题

反向传播算法的工作原理是从输出层到输入层，传播误差的梯度。 一旦该算法已经计算了网络中每个参数的损失函数的梯度，它就使用这些梯度来用梯度下降步骤来更新每个参数。

不幸的是，梯度往往变得越来越小，随着算法进展到较低层。	结果，梯度下降更新使得低层连接权重实际上保持不变，并且训练永远不会收敛到良好的解决方案。这被称为 __梯度消失问题__。	

在某些情况下，可能会发生相反的情况：梯度可能变得越来越大，许多层得到了非常大 的权重更新，算法发散。这是 __梯度爆炸的问题__。


In [3]:
import tensorflow as tf
import numpy as np

###  减缓这一问题的方法一：Xavier初始化和He初始化

提出需要保持每一层的输入和输出的方差一致，并且需要在反向流动过某一层时，前后的方差也要一致。

当输入连接的数量大致等于输出连接的数量时，可以得到更简单的等式：(就是第十章中用到的方差)

</br>
<div align=center><img width="400" height="300" src="./static/1.jpg"/></div>

该方法的折中方案公式为Xavier初始化。

ReLU激活函数的初始化方法有时称为He初始化。

#### He 初始化
He	初始化只考虑了扇入，而不是像	Xavier	初始化那样扇入和扇出之间的平均值。	

这也 是	`variance_scaling_initializer()`	函数的默认值，但您可以通过设置参 数	`mode	="FAN_AVG"`	来更改它。


In [4]:
# he_init = tf.contrib.layers.variance_scaling_initializer()
# hidden1 = tf.layers.dense(X,n_hidden,activation=tf.nn.relu,kernel_initializer=he_init,name='hidden1')

###  减缓这一问题的方法二：更换激活函数

一般来说	ELU	>	leaky	ReLU（及其变体）>	ReLU	>	tanh	>	sigmoid

#### elu 激活函数

tensorflow 中提供了 elu 函数用于建立神经网络：

In [5]:
# hidden1 = tf.layers.dense(x,n_hidden,activation=tf.nn.elu,name='hidden1')

#### leaky	ReLU 激活函数

tensorflow 中没有针对 leaky	ReLU 的函数，但是可以自己定义：

In [6]:
# def leak_relu(z,name=None):
#     return tf.maximum(0.01 * z,z,name = name)
# hidden1 = tf.layers.dense(x,n_hidden1,activation=leak_relu,name='hidden1')

###  减缓这一问题的方法三：批量标准化

尽管使用	He初始化和	ELU（或任何	ReLU	变体）可以显著减少训练开始阶段的梯度消失/爆炸问题，但不保证在训练期间问题不会回来。


__该技术会在每一层激活函数之前在模型中介入一个操作，操作实现简单零中心化和归一化输入，之后再通过每层的两个新参数（一个缩放，一个移动）来控制缩放和移动的结果。这样的操作会让模型学会最佳规模和每层输入的平均值。__
</br>
<div align=center><img width="400" height="300" src="./static/2.jpg"/></div>
<div align=center><img width="660" height="600" src="./static/3.jpg"/></div>

在测试时，没有小批量计算经验均值和标准差，所以您只需使用整个训练集的均值和标准 差。	这些通常在训练期间使用移动平均值进行有效计算。	因此，总的来说，每个批次标准化 的层次都学习了四个参数：	`γ（标度）`，	`β（偏移）`，	`μ（平均值）` 和	`σ（标准差）`。

#### 使用 BN 的优缺点

再使用饱和激活函数的深度神经网络中，批量归一化取得了非常好的成绩，而且还会为降低后续的正则化的技术需求。

但BN的使用确实也增加了模型的复杂度，降低了网络额速度。

#### 使用	TensorFlow	实现批量标准化

在 functools 模块中有一个工具partial()，可以用来"冻结"一个函数的参数，并返回"冻结"参数后的新函数。

In [7]:
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data

#### tf.GraphKeys.UPDATE_OPS  

`batch normalization` 的兩個重要的參數，`moving_mean` 和 `moving_var`,两个 `batch_normalization` 中更新 `mean` 和 `variance` 的操作，需要保证它们在train_op前完成。

这两个操作是在 `tensorflow` 的内部实现中自动被加入 `tf.GraphKeys.UPDATE_OPS` 这个集合的，在 `tf.contrib.layers.batch_norm` 的参数中可以看到有一项`updates_collections` 的默认值即为 `tf.GraphKeys.UPDATE_OPS` ，而在 `tf.layers.batch_normalization` 中则是直接将两个更新操作放入了上述集合。

#### 小结：

1、如果不通过 `tf.get_collection` 来获取，`moving_mean` 和 `moving_var` 不会更新，一直都会是初始值。

2、当然 如果进行批量标准化更新参数，`tf.layers.batch_normalization` 中的  `training` 要设置为 `True` : `tf.placeholder_with_default(True, (), name='training')`

https://ithelp.ithome.com.tw/articles/10220410

https://github.com/jason9075/ithome_tensorflow_series/blob/day17/17/update_op.py


In [23]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

mnist = input_data.read_data_sets('./mldata/')
batch_norm_momentum = 0.9
learning_rate = 0.01

X = tf.placeholder(tf.float32,shape=(None,n_inputs),name = 'X')
y = tf.placeholder(tf.int64,shape=None,name='y')

training = tf.placeholder_with_default(False,shape=(),name = 'training') #给Batch	norm加一个placeholder 

with tf.name_scope('dnn'):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    
    my_batch_norm_layer = partial(
        tf.layers.batch_normalization,
        training = training,
        momentum = batch_norm_momentum
    )

    my_dense_layer = partial(
        tf.layers.dense,
        kernel_initializer = he_init
    )
    
    hidden1 = my_dense_layer(X,n_hidden1,name='hidden1')
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    
    hidden2 = my_dense_layer(bn1,n_hidden2,name='hidden2')
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    
    logits_before_bn = my_dense_layer(bn2,n_outputs,name='outputs')
    logits = my_batch_norm_layer(logits_before_bn)
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y,logits=logits)
    loss = tf.reduce_mean(xentropy,name='loss')

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 200

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) 

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        for iteraction in range(mnist.train.num_examples // batch_size):
            X_batch,y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op,extra_update_ops],feed_dict={training:True,X:X_batch,y:y_batch})
        
        accuracy_val = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch, 'Test accuracy:', accuracy_val)



Extracting ./mldata/train-images-idx3-ubyte.gz
Extracting ./mldata/train-labels-idx1-ubyte.gz
Extracting ./mldata/t10k-images-idx3-ubyte.gz
Extracting ./mldata/t10k-labels-idx1-ubyte.gz
0 Test accuracy: 0.8723
1 Test accuracy: 0.8956
2 Test accuracy: 0.9121
3 Test accuracy: 0.9206
4 Test accuracy: 0.9289
5 Test accuracy: 0.9351
6 Test accuracy: 0.9395
7 Test accuracy: 0.9428
8 Test accuracy: 0.9473
9 Test accuracy: 0.949
10 Test accuracy: 0.9521
11 Test accuracy: 0.9518
12 Test accuracy: 0.9559
13 Test accuracy: 0.9584
14 Test accuracy: 0.9585
15 Test accuracy: 0.9609
16 Test accuracy: 0.963
17 Test accuracy: 0.9641
18 Test accuracy: 0.9647
19 Test accuracy: 0.9664


#### 或者将上面的 `train` 改写成：

In [25]:
# with tf.name_scope('train'):
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#     extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#     with tf.control_dependencies(extra_update_ops):
#         training_op = optimizer.minimize(loss)


# with tf.Session() as sess:
#     init.run()

#     for epoch in range(n_epochs):
#         for iteraction in range(mnist.train.num_examples // batch_size):
#             X_batch,y_batch = mnist.train.next_batch(batch_size)
#             sess.run(training_op,feed_dict={training:True,X:X_batch,y:y_batch})
        
#         accuracy_val = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
#         print(epoch, 'Test accuracy:', accuracy_val)

 #### 这样，你只需要在训练过程中评估 training_op，TensorFlow 也会自动运行更新操作

In [24]:
# tf.reset_default_graph()

# n_inputs = 28 * 28
# n_hidden1 = 300
# n_hidden2 = 100
# n_outputs = 10

# mnist = input_data.read_data_sets('./mldata/')
# batch_norm_momentum = 0.9
# learning_rate = 0.01

# X = tf.placeholder(tf.float32,shape=(None,n_inputs),name = 'X')
# y = tf.placeholder(tf.int64,shape=None,name='y')

# training = tf.placeholder_with_default(False,shape=(),name = 'training') #给Batch	norm加一个placeholder

# with tf.name_scope('dnn'):
#     he_init = tf.contrib.layers.variance_scaling_initializer()
    
#     my_batch_norm_layer = partial(
#         tf.layers.batch_normalization,
#         training = training,
#         momentum = batch_norm_momentum
#     )

#     my_dense_layer = partial(
#         tf.layers.dense,
#         kernel_initializer = he_init
#     )
    
#     hidden1 = my_dense_layer(X,n_hidden1,name='hidden1')
#     bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    
#     hidden2 = my_dense_layer(bn1,n_hidden2,name='hidden2')
#     bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    
#     logits_before_bn = my_dense_layer(bn2,n_outputs,name='outputs')
#     logits = my_batch_norm_layer(logits_before_bn)
    
# with tf.name_scope('loss'):
#     xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y,logits=logits)
#     loss = tf.reduce_mean(xentropy,name='loss')

# with tf.name_scope('train'):
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#     training_op = optimizer.minimize(loss)
    
# with tf.name_scope('eval'):
#     correct = tf.nn.in_top_k(logits,y,1)
#     accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

# init = tf.global_variables_initializer()
# saver = tf.train.Saver()

# n_epochs = 20
# batch_size = 200

# with tf.Session() as sess:
#     init.run()

#     for epoch in range(n_epochs):
#         for iteraction in range(mnist.train.num_examples // batch_size):
#             X_batch,y_batch = mnist.train.next_batch(batch_size)
#             sess.run([training_op],feed_dict={training:True,X:X_batch,y:y_batch})
        
#         accuracy_val = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
#         print(epoch, 'Test accuracy:', accuracy_val)


Extracting ./mldata/train-images-idx3-ubyte.gz
Extracting ./mldata/train-labels-idx1-ubyte.gz
Extracting ./mldata/t10k-images-idx3-ubyte.gz
Extracting ./mldata/t10k-labels-idx1-ubyte.gz
0 Test accuracy: 0.7275
1 Test accuracy: 0.762
2 Test accuracy: 0.7851
3 Test accuracy: 0.8003
4 Test accuracy: 0.8213
5 Test accuracy: 0.8446
6 Test accuracy: 0.8438
7 Test accuracy: 0.8652
8 Test accuracy: 0.8791
9 Test accuracy: 0.8897
10 Test accuracy: 0.893
11 Test accuracy: 0.8897
12 Test accuracy: 0.8905
13 Test accuracy: 0.8996
14 Test accuracy: 0.8976
15 Test accuracy: 0.9018
16 Test accuracy: 0.9019
17 Test accuracy: 0.8985
18 Test accuracy: 0.905
19 Test accuracy: 0.9064


In [29]:
import tensorflow as tf
a_1 = tf.Variable(1)
b_1 = tf.Variable(2)
update_op = tf.assign(a_1, 10)
add = tf.add(a_1, b_1)

a_2 = tf.Variable(1)
b_2 = tf.Variable(2)
update_op = tf.assign(a_2, 10)
with tf.control_dependencies([update_op]):
    add_with_dependencies = tf.add(a_2, b_2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ans_1, ans_2 = sess.run([add, add_with_dependencies])
    print("Add: ", ans_1)
    print("Add_with_dependency: ", ans_2)

Add:  3
Add_with_dependency:  12


###  减缓这一问题的方法四：梯度裁剪

减少梯度爆炸问题的一种常用技术是在反向传播过程中简单地剪切梯度，使它们不超过某个 __阈值__

一般 来说，人们更喜欢 __批量标准化__ ，但了解 __梯度裁剪__ 以及如何实现它仍然是有用的。

在 TensorFlow 中，优化器的	`minimize()`	函数负责计算梯度并应用它们，所以您必须首先调用优化器的	`compute_gradients()` 方法，然后使用	`clip_by_value()` 函数创建一个 `裁剪梯度` 的 操作，最后创建一个操作来使用优化器的 `apply_gradients()`	方法应用裁剪梯度：


In [ ]:
# threshold = 1.0

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# grad_and_var = optimizer.compute_gradients(loss)
# capped_gvs =  [(tf.clip_by_value(grad, -threshold, threshold), var)for grad, var in grads_and_vars] 
# training_op = optimizer.apply_gradients(capped_gvs)

像往常一样，您将在每个训练阶段运行这个	training_op	。	它将计算梯度，将它们裁剪到 -1.0	和	1.0	之间，并应用它们。		threhold	是您可以调整的超参数。


### 复用预训练层


从零开始训练一个非常大的	DNN	通常不是一个好主意，相反，__您应该总是尝试找到一个现有 的神经网络来完成与您正在尝试解决的任务类似的任务，然后复用这个网络的较低层：这就 是所谓的迁移学习__ 。这不仅会大大加快训练速度，还将需要更少的训练数据。


#### 比如现在有这样一个 tensorflow 模型：

In [65]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50

n_outputs = 10


X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y = tf.placeholder(tf.int32,shape=None,name='y')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X,n_hidden1,activation=tf.nn.relu,name='hidden1')
    hidden2 = tf.layers.dense(hidden1,n_hidden2,activation=tf.nn.relu,name='hidden2')
    hidden3 = tf.layers.dense(hidden2,n_hidden3,activation=tf.nn.relu,name='hidden3')
    hidden4 = tf.layers.dense(hidden3,n_hidden4,activation=tf.nn.relu,name='hidden4')
    hidden5 = tf.layers.dense(hidden4,n_hidden5,activation=tf.nn.relu,name='hidden5')

    logits = tf.layers.dense(hidden5,n_outputs,name='outputs')

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy,name='loss')
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32),name='accuracy')

learning_rate = 0.01
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs =  [(tf.clip_by_value(grad, -threshold, threshold), var)for grad, var in grads_and_vars] 
training_op = optimizer.apply_gradients(capped_gvs)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteraction in range(mnist.train.num_examples // batch_size):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op],feed_dict={X:x_batch,y:y_batch})
        accuracy_val = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print('accuracy_val:',accuracy_val)
    save_path = saver.save(sess,'./my_model_final.ckep')

accuracy_val: 0.8876
accuracy_val: 0.9209
accuracy_val: 0.9393
accuracy_val: 0.9506
accuracy_val: 0.9533
accuracy_val: 0.9597
accuracy_val: 0.9644
accuracy_val: 0.9642
accuracy_val: 0.9662
accuracy_val: 0.9692
accuracy_val: 0.9698
accuracy_val: 0.9625
accuracy_val: 0.9697
accuracy_val: 0.972
accuracy_val: 0.9724
accuracy_val: 0.9737
accuracy_val: 0.9712
accuracy_val: 0.972
accuracy_val: 0.9723
accuracy_val: 0.9733


接下来我们要复用之前模型的前几层

In [68]:
import os

In [69]:
ckpt = tf.train.get_checkpoint_state(os.path.dirname(r"./"))
ckpt

model_checkpoint_path: ".\\my_model_final.ckep"
all_model_checkpoint_paths: ".\\my_model_final.ckep"

In [73]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused

n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y = tf.placeholder(tf.int32,shape=None,name='y')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X,n_hidden1,activation=tf.nn.relu,name='hidden1')
    hidden2 = tf.layers.dense(hidden1,n_hidden2,activation=tf.nn.relu,name='hidden2')
    hidden3 = tf.layers.dense(hidden2,n_hidden3,activation=tf.nn.relu,name='hidden3')
    hidden4 = tf.layers.dense(hidden3,n_hidden4,activation=tf.nn.relu,name='hidden4')

    logits = tf.layers.dense(hidden4,n_outputs,name='outputs')

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy,name='loss')

    
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32),name='accuracy')

[...] # build new model with the same definition as before for hidden layers 1-3

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='hidden[123]')
reuse_vars_dict = dict([(var.op.name,var) for var in reuse_vars]) 
restore_saver = tf.train.Saver(reuse_vars_dict) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    
    init.run()
    restore_saver.restore(sess,'.\\my_model_final.ckep')
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:x_batch,y:y_batch})
        accuracy_val = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,'accuracy_val: ',accuracy_val)
        
    saver_path = saver.save(sess,'./my_new_model_final.ckep')

INFO:tensorflow:Restoring parameters from .\my_model_final.ckep
0 accuracy_val:  0.9543
1 accuracy_val:  0.9648
2 accuracy_val:  0.9696
3 accuracy_val:  0.9706
4 accuracy_val:  0.9725
5 accuracy_val:  0.9718
6 accuracy_val:  0.9739
7 accuracy_val:  0.9742
8 accuracy_val:  0.9733
9 accuracy_val:  0.9743
10 accuracy_val:  0.9743
11 accuracy_val:  0.9748
12 accuracy_val:  0.9738
13 accuracy_val:  0.9759
14 accuracy_val:  0.9744
15 accuracy_val:  0.9745
16 accuracy_val:  0.9754
17 accuracy_val:  0.975
18 accuracy_val:  0.9742
19 accuracy_val:  0.9747


首先我们建立新的模型，确保复制原始模型的隐藏层	1	到	3。我们还创建一个节点来初始化 所有变量。	然后我们得到刚刚用	trainable	=	True （这是默认值）创建的所有变量的列表， 我们只保留那些范围与正则表达式 hidden [123] 相匹配的变量（即，我们得到所有可训练的隐藏层 1 到 3 中的变量）。	

接下来，我们创建一个字典，将原始模型中每个变量的名称映射 到新模型中的名称（通常需要保持完全相同的名称）。	然后，我们创建一个	Saver	，它将只 恢复这些变量，并且创建另一个	Saver	来保存整个新模型，而不仅仅是第	1	层到第	3	层。然 后，我们开始一个会话并初始化模型中的所有变量，然后从原始模型的层	1	到	3中恢复变量 值。最后，我们在新任务上训练模型并保存。




## 更快的优化器

训练一个非常大的深度神经网络可能会非常缓慢。	

到目前为止，我们已经看到了四种加速训练的方法（并且达到更好的解决方案）：

* __1、对连接权重应用良好的初始化策略__，


* __2、使用良好的激活函数__，


* __3、使用批量规范化__


* __4、重用预训练网络__

另一个巨大的速度提升来自使用比普通渐变下降优化器更快的优化器。	在本节中，我们将介绍最流行的：__动量优化__ ，__Nesterov	加速梯度__，__AdaGrad__，__RMSProp__，最后是	__Adam	优化__。

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

#### 本节的结论是:

您几乎总是应该使用	__Adam_optimization__	，所以如果您不关心它是如何 工作的，只需使用	__AdamOptimizer__	替换您的	__GradientDescentOptimizer__	

### 学习率调整

找到一个好的学习速度可能会非常棘手。	如果设置太高，训练实际上可能偏离。	如果设置得太低，训练最终会收敛到最佳状态，但这需要很长时间。	

我们可以使用 __指数调度__  

<div align=center><img width="600" height="400" src="./static/4.jpg"/></div>

In [ ]:
# initial_learning_rate = 0.1
# decay_steps = 10000
# decay_rate =1/10

# gloable_step = tf.Variable(0,trainable=False,name='gloable_step')
# learning_rate = tf.train.exponential_decay(initial_learning_rate,gloable_step,decay_steps,decay_rate)

# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
# training_op = optimizer.minimize(loss,global_step=gloable_step)

使用	TensorFlow	的	__exponential_decay()__	函数来定义指数衰减的学习 率（	η0=	0.1	和	r	=	10,000	）

接下来，我们使用这个衰减的学习率创建一个优化器（在这 个例子中是一个	__MomentumOptimizer__	）。	最后，我们通过调用优化器的	__minimize()__	方法来创 建训练操作；因为我们将	__global_step__	变量传递给它，所以请注意增加它。

#### 由于	AdaGrad，RMSProp	和	Adam	优化自动降低了训练期间的学习率，因此不需要添加额 外的学习率调整。	对于其他优化算法，使用指数衰减或性能调度可显著加速收敛。


In [74]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50

n_outputs = 10


X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y = tf.placeholder(tf.int32,shape=None,name='y')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X,n_hidden1,activation=tf.nn.relu,name='hidden1')
    hidden2 = tf.layers.dense(hidden1,n_hidden2,activation=tf.nn.relu,name='hidden2')
    hidden3 = tf.layers.dense(hidden2,n_hidden3,activation=tf.nn.relu,name='hidden3')
    hidden4 = tf.layers.dense(hidden3,n_hidden4,activation=tf.nn.relu,name='hidden4')
    hidden5 = tf.layers.dense(hidden4,n_hidden5,activation=tf.nn.relu,name='hidden5')

    logits = tf.layers.dense(hidden5,n_outputs,name='outputs')

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy,name='loss')
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32),name='accuracy')

learning_rate = 0.01
threshold = 1.0
with tf.name_scope('train'):
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate =1/10

    gloable_step = tf.Variable(0,trainable=False,name='gloable_step')
    learning_rate = tf.train.exponential_decay(initial_learning_rate,gloable_step,decay_steps,decay_rate)

    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
    training_op = optimizer.minimize(loss,global_step=gloable_step)


init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteraction in range(mnist.train.num_examples // batch_size):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op],feed_dict={X:x_batch,y:y_batch})
        accuracy_val = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print('accuracy_val:',accuracy_val)
    save_path = saver.save(sess,'./my_model_final.ckep')

accuracy_val: 0.9507
accuracy_val: 0.9654
accuracy_val: 0.971
accuracy_val: 0.9729
accuracy_val: 0.9766
accuracy_val: 0.9779
accuracy_val: 0.9787
accuracy_val: 0.9792
accuracy_val: 0.9804
accuracy_val: 0.9805
accuracy_val: 0.9799
accuracy_val: 0.9804
accuracy_val: 0.9812
accuracy_val: 0.9808
accuracy_val: 0.9807
accuracy_val: 0.9807
accuracy_val: 0.9808
accuracy_val: 0.9807
accuracy_val: 0.9807
accuracy_val: 0.9807


## 通过正则化避免过拟合

有了数以百万计的参数，你可以适应整个动物园。	在本节中，我们将介绍一些最流行的神经 网络正则化技术，以及如何用	__TensorFlow__	实现它们：

* __早期停止__

* __l1	和	l2	正则化__

* __drop	out__

* __最大范数正则化__

* __数据增强__

### 早期停止

与	TensorFlow	实现方法之一是评估其对设置定期（例如，每	50	步）验证模型，并保存一 个“winner”的快照，如果它优于以前“winner”的快照。计算自上次“winner”快照保存以来的步 数，并在达到某个限制时（例如	2000	步）中断训练。	然后恢复最后的“winner”快照。

虽然早期停止在实践中运行良好，但是通过将其与其他正则化技术相结合，您通常可以在网 络中获得更高的性能。


### L1	和	L2	正则化

你可以使用	l1	和	l2	正则化约束一个神经网络 的连接权重（但通常不是它的偏置）

In [83]:
tf.reset_default_graph()

In [84]:
n_inputs = 28 * 28  #  MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [85]:
scale = 0.01

In [86]:
my_dense_layer = partial(
    tf.layers.dense,activation=tf.nn.relu,kernel_regularizer=tf.contrib.layers.l1_regularizer(scale)
)

with tf.name_scope('dnn'):
    hidden1 = my_dense_layer(X,n_hidden1,name='hidden1')
    hidden2 = my_dense_layer(hidden1,n_hidden2,name='hidden2')    
    logits = my_dense_layer(hidden2,n_outputs,name='outputs',activation=None)    

该代码创建了一个具有两个隐藏层和一个输出层的神经网络，并且还在图中创建节点以计算 与每个层的权重相对应的	l1	正则化损失。

TensorFlow	会自动将这些节点添加到包含所有正则 化损失的特殊集合中。	

您只需要将这些正则化损失添加到您的整体损失中:

In [87]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    base_loss = tf.reduce_mean(xentropy,name = 'avg_xentropy')
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_loss,name='loss')

In [88]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### 查看 base_loss 和 base_loss 和 loss

In [117]:
# n_epochs = 2
# batch_size = 2
# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs):
#         for iteration in range(mnist.train.num_examples // batch_size):
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
#             print('base_loss',sess.run(base_loss, feed_dict={X: X_batch, y: y_batch}))
#             print('base_loss',sess.run(reg_loss))
#             print('loss:',sess.run(loss, feed_dict={X: X_batch, y: y_batch}))


#         accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
#                                                 y: mnist.test.labels})
#         print(epoch, "Test accuracy:", accuracy_val)


base_loss 2.0395257
reg_loss [87.28086, 9.8397045, 0.76513356]
loss: 99.92523
base_loss 1.9679055
reg_loss [87.04675, 9.824848, 0.7644928]
loss: 99.604004
base_loss 1.5994551
reg_loss [86.813736, 9.810329, 0.7641037]
loss: 98.98763
base_loss 1.7554479
reg_loss [86.58022, 9.795431, 0.76366484]
loss: 98.89477
base_loss 1.5769806
reg_loss [86.34713, 9.78064, 0.76318014]
loss: 98.46793
base_loss 1.890858
reg_loss [86.11391, 9.765747, 0.7626589]
loss: 98.53317
base_loss 1.5533714
reg_loss [85.88197, 9.751421, 0.7623978]
loss: 97.949165
base_loss 2.174447
reg_loss [85.64851, 9.736156, 0.7617344]
loss: 98.32085
base_loss 2.086952
reg_loss [85.41607, 9.721332, 0.7612041]
loss: 97.98556
base_loss 1.9674599
reg_loss [85.18447, 9.706643, 0.7607554]
loss: 97.61933
base_loss 1.8041854
reg_loss [84.95316, 9.692038, 0.7603098]
loss: 97.209694
base_loss 2.1641507
reg_loss [84.72172, 9.676592, 0.7596122]
loss: 97.322075
base_loss 1.9816387
reg_loss [84.490944, 9.661867, 0.7591539]
loss: 96.8936
base_lo

base_loss 1.8778477
reg_loss [63.234467, 8.264755, 0.71789557]
loss: 74.09496
base_loss 1.5407398
reg_loss [63.033348, 8.251349, 0.71772176]
loss: 73.54316
base_loss 1.78876
reg_loss [62.837227, 8.238059, 0.71733105]
loss: 73.581375
base_loss 1.5401347
reg_loss [62.640957, 8.224937, 0.71706086]
loss: 73.12309
base_loss 1.7979263
reg_loss [62.442364, 8.211532, 0.71661276]
loss: 73.168434
base_loss 1.4467611
reg_loss [62.245735, 8.198847, 0.71641266]
loss: 72.60776
base_loss 1.6996384
reg_loss [62.04963, 8.185421, 0.7159836]
loss: 72.65067
base_loss 1.6148676
reg_loss [61.854607, 8.172226, 0.7156759]
loss: 72.357376
base_loss 1.5372995
reg_loss [61.656727, 8.159006, 0.7154152]
loss: 72.06845
base_loss 2.026428
reg_loss [61.45916, 8.144889, 0.715031]
loss: 72.34551
base_loss 1.7039036
reg_loss [61.26361, 8.131602, 0.7147338]
loss: 71.81385
base_loss 1.8413584
reg_loss [61.06515, 8.118069, 0.7144553]
loss: 71.73904
base_loss 1.5661857
reg_loss [60.873962, 8.104946, 0.7140274]
loss: 71.2591

base_loss 1.9265676
reg_loss [40.877586, 6.695016, 0.6847508]
loss: 50.183918
base_loss 1.4810266
reg_loss [40.718178, 6.6835217, 0.68453026]
loss: 49.567253
base_loss 1.533439
reg_loss [40.559494, 6.6719184, 0.68428564]
loss: 49.449135
base_loss 1.3547475
reg_loss [40.402264, 6.660348, 0.68409336]
loss: 49.10145
base_loss 0.99241495
reg_loss [40.247246, 6.6490245, 0.68397033]
loss: 48.57266
base_loss 1.2234796
reg_loss [40.083923, 6.6370907, 0.6838158]
loss: 48.62831
base_loss 0.5786583
reg_loss [39.92995, 6.6261225, 0.68379897]
loss: 47.81853
base_loss 1.301538
reg_loss [39.77377, 6.6150427, 0.6836208]
loss: 48.37397
base_loss 1.529404
reg_loss [39.61509, 6.6031027, 0.68342614]
loss: 48.431023
base_loss 1.1555587
reg_loss [39.45929, 6.5913305, 0.6832816]
loss: 47.88946
base_loss 1.5562452
reg_loss [39.307068, 6.579856, 0.683287]
loss: 48.126457
base_loss 1.0889202
reg_loss [39.14976, 6.567872, 0.682953]
loss: 47.489506
base_loss 0.9611114
reg_loss [38.99538, 6.556666, 0.6826821]
loss

base_loss 1.3803706
reg_loss [18.985016, 4.9248796, 0.6538212]
loss: 25.944088
base_loss 1.1639845
reg_loss [18.885237, 4.9157186, 0.65387714]
loss: 25.618816
base_loss 1.5805942
reg_loss [18.786331, 4.907072, 0.65362555]
loss: 25.927624
base_loss 1.5145345
reg_loss [18.684607, 4.89793, 0.6535637]
loss: 25.750635
base_loss 1.7860909
reg_loss [18.584171, 4.8869023, 0.6531621]
loss: 25.910326
base_loss 1.2528584
reg_loss [18.467644, 4.876505, 0.6530301]
loss: 25.250038
base_loss 1.6540859
reg_loss [18.35514, 4.8661246, 0.6529147]
loss: 25.528265
base_loss 1.4359901
reg_loss [18.246748, 4.8565226, 0.652815]
loss: 25.192076
base_loss 1.4779046
reg_loss [18.13815, 4.846445, 0.6527037]
loss: 25.115204
base_loss 1.3662028
reg_loss [18.035997, 4.8368735, 0.6525932]
loss: 24.891668
base_loss 1.498264
reg_loss [17.928102, 4.826585, 0.65241224]
loss: 24.905365
base_loss 1.126251
reg_loss [17.827549, 4.8170247, 0.6523669]
loss: 24.423191
base_loss 1.190973
reg_loss [17.724438, 4.80797, 0.6524318]


base_loss 1.7955225
reg_loss [8.418856, 3.8251708, 0.6350934]
loss: 14.674643
base_loss 1.3575146
reg_loss [8.347444, 3.8163095, 0.6350903]
loss: 14.156357
base_loss 1.6069613
reg_loss [8.271088, 3.8075294, 0.63496727]
loss: 14.320545
base_loss 1.4952552
reg_loss [8.196586, 3.798268, 0.63460416]
loss: 14.124714
base_loss 0.89845777
reg_loss [8.13836, 3.7906947, 0.6344931]
loss: 13.462006
base_loss 0.8526445
reg_loss [8.068884, 3.782728, 0.6342827]
loss: 13.33854
base_loss 1.5826192
reg_loss [8.005368, 3.7741416, 0.63410485]
loss: 13.996234
base_loss 1.464217
reg_loss [7.952378, 3.7669525, 0.63394195]
loss: 13.817489
base_loss 1.8113596
reg_loss [7.8747363, 3.7570512, 0.63366944]
loss: 14.0768175
base_loss 1.5367124
reg_loss [7.8027735, 3.7478387, 0.63337636]
loss: 13.720701
base_loss 0.71357846
reg_loss [7.761671, 3.7425337, 0.6333013]
loss: 12.851085
base_loss 1.6236383
reg_loss [7.695455, 3.734332, 0.6331821]
loss: 13.686608
base_loss 1.4033254
reg_loss [7.6336117, 3.7258215, 0.63316

...

reg_loss [0.51477486, 0.16694634, 0.23502082]
loss: 2.1090195
base_loss 0.39074373
reg_loss [0.5155661, 0.16709217, 0.2354265]
loss: 1.3088286
base_loss 0.4215581
reg_loss [0.51705134, 0.16758512, 0.23503931]
loss: 1.341234
base_loss 1.0929835
reg_loss [0.5182165, 0.16738722, 0.23511319]
loss: 2.0137005
base_loss 0.156153
reg_loss [0.5177807, 0.16688131, 0.23478536]
loss: 1.0756004
base_loss 0.5881361
reg_loss [0.5133299, 0.16665445, 0.23476121]
loss: 1.5028816
base_loss 0.2510268
reg_loss [0.51345575, 0.16670392, 0.23503536]
loss: 1.1662219
base_loss 0.3797362
reg_loss [0.51177514, 0.16675638, 0.23512565]
loss: 1.2933934
base_loss 0.46136415
reg_loss [0.51185507, 0.16668144, 0.23541065]
loss: 1.3753114
base_loss 1.0609695
reg_loss [0.51100665, 0.16700748, 0.2351288]
loss: 1.9741124
base_loss 0.6580682
reg_loss [0.51318586, 0.16667828, 0.23467351]
loss: 1.5726058
base_loss 0.4376668
reg_loss [0.51247245, 0.16678712, 0.23495044]
loss: 1.3518769
base_loss 0.53864825
reg_loss [0.5135507, 

base_loss 0.13822812
reg_loss [0.5009289, 0.16660921, 0.23418584]
loss: 1.039952
base_loss 0.20488386
reg_loss [0.5020807, 0.16654822, 0.23429468]
loss: 1.1078074
base_loss 0.54559964
reg_loss [0.5032601, 0.16711639, 0.23464067]
loss: 1.4506168
base_loss 0.48621428
reg_loss [0.50140476, 0.16672558, 0.23516297]
loss: 1.3895075
base_loss 0.32802245
reg_loss [0.5007965, 0.1669429, 0.2350514]
loss: 1.2308133
base_loss 0.24273553
reg_loss [0.501585, 0.16704777, 0.23510595]
loss: 1.1464742
base_loss 0.089001894
reg_loss [0.49991575, 0.1670726, 0.23502344]
loss: 0.99101365
base_loss 0.23412564
reg_loss [0.50051874, 0.16692086, 0.235524]
loss: 1.1370893
base_loss 0.2550875
reg_loss [0.4989389, 0.16700904, 0.23535188]
loss: 1.1563873
base_loss 0.45588502
reg_loss [0.50321835, 0.16727024, 0.23510955]
loss: 1.3614831
base_loss 0.5156016
reg_loss [0.502324, 0.16723183, 0.23525886]
loss: 1.4204161
base_loss 0.5717395
reg_loss [0.5016035, 0.16704643, 0.23479329]
loss: 1.4751828
base_loss 0.23758551


base_loss 0.050011665
reg_loss [0.50922513, 0.16774437, 0.23412167]
loss: 0.96110284
base_loss 0.57759225
reg_loss [0.50982416, 0.16755188, 0.23484074]
loss: 1.489809
base_loss 0.19414058
reg_loss [0.5074449, 0.16780153, 0.23468156]
loss: 1.1040686
base_loss 0.14896753
reg_loss [0.50676674, 0.1676448, 0.23462033]
loss: 1.0579994
base_loss 1.0851738
reg_loss [0.50533414, 0.16759683, 0.23491965]
loss: 1.9930245
base_loss 0.113181666
reg_loss [0.50665486, 0.16751693, 0.23458993]
loss: 1.0219433
base_loss 0.1801562
reg_loss [0.50543183, 0.16776928, 0.23467107]
loss: 1.0880284
base_loss 0.46105826
reg_loss [0.50801504, 0.16756462, 0.23523101]
loss: 1.371869
base_loss 1.0825015
reg_loss [0.5057318, 0.16780166, 0.23550391]
loss: 1.9915389
base_loss 0.2653829
reg_loss [0.50599474, 0.1673529, 0.23549406]
loss: 1.1742246
base_loss 0.39108092
reg_loss [0.50656015, 0.16759492, 0.23570739]
loss: 1.3009434
base_loss 1.2404711
reg_loss [0.5067138, 0.16724329, 0.23600566]
loss: 2.150434
base_loss 0.17

loss: 1.0699363
base_loss 0.4043027
reg_loss [0.5039637, 0.16791959, 0.23670477]
loss: 1.3128908
base_loss 0.25942343
reg_loss [0.50449955, 0.16719694, 0.236946]
loss: 1.1680659
base_loss 0.26866713
reg_loss [0.5043787, 0.16704273, 0.23671068]
loss: 1.1767992
base_loss 0.28448188
reg_loss [0.5038854, 0.16672052, 0.23646906]
loss: 1.1915568
base_loss 0.7754294
reg_loss [0.5033293, 0.1669849, 0.23615395]
loss: 1.6818976
base_loss 0.12541161
reg_loss [0.50271237, 0.16666943, 0.23623042]
loss: 1.0310239
base_loss 0.24342063
reg_loss [0.5005274, 0.16696759, 0.23618259]
loss: 1.1470982
base_loss 0.13835749
reg_loss [0.50012803, 0.16665165, 0.23627214]
loss: 1.0414094
base_loss 0.15931289
reg_loss [0.49854463, 0.16690405, 0.23587242]
loss: 1.060634
base_loss 0.6057719
reg_loss [0.5047383, 0.167224, 0.23631988]
loss: 1.5140541
base_loss 0.11036171
reg_loss [0.50291586, 0.16697899, 0.23628883]
loss: 1.0165453
base_loss 0.4468922
reg_loss [0.50223243, 0.16679072, 0.23637901]
loss: 1.3522943
base

KeyboardInterrupt: 

In [89]:
n_epochs = 20
batch_size = 200
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Test accuracy: 0.7471
1 Test accuracy: 0.736
2 Test accuracy: 0.5298
3 Test accuracy: 0.4616
4 Test accuracy: 0.4692
5 Test accuracy: 0.5141
6 Test accuracy: 0.5543
7 Test accuracy: 0.584
8 Test accuracy: 0.6152
9 Test accuracy: 0.639
10 Test accuracy: 0.6619
11 Test accuracy: 0.6817
12 Test accuracy: 0.6903
13 Test accuracy: 0.709
14 Test accuracy: 0.7162
15 Test accuracy: 0.7263
16 Test accuracy: 0.7347
17 Test accuracy: 0.7469
18 Test accuracy: 0.7527
19 Test accuracy: 0.7565


### 正则化： Dropout

Dropout什么意思呢？

因为我们的参数很多，在每次迭代训练的时候我们并不是希望神一个神经元都是被激活的，我们曾经通过激活函数(如果激活值小鱼阈值则不会传递到下一层)，现在我们也可以通过dorpout来做这样的事情，他就是以一定的概率值随机丢弃一些神经元，单这些丢弃的神经元只是在这一次迭代中不参与训练，下一次训练我们还可以通过Dorpout在进行丢弃。

这样一来，我们这次训练的时候就不是所有的神经元都参与训练。

以我们的图为例，我们可以看到输入输出神经元都有四个，如果我们使用dropout，那么我们输入神经元的第一个和第三个神经元就不在参与训练，这样使得我们的模型变得简单，减少了模型的参数。降低复杂度。同时随机的神经元进行组合，减少了神经元之间可能形成的共同依赖，dropout的神经网络是由dropout之后的子模型组成的，因为我们在训练完之后，我们是需要所有的神经元参与的(我们在训练的时候训练很多子模型，但是最后我们要将训练好的子模型进行一个组合，因为我们是需要所有的神经元参与预测) 这样就有利于提高我们模型的泛化能力

In [121]:
tf.reset_default_graph()

In [122]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [124]:
training = tf.placeholder_with_default(False,shape=(),name='train')

dropout_rate = 0.5
x_drop = tf.layers.dropout(X,dropout_rate,training=training)

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(x_drop,n_hidden1,activation=tf.nn.relu,name='hidden1')
    hidden1_drop = tf.layers.dropout(hidden1,dropout_rate,training=training)
    
    hidden2 = tf.layers.dense(hidden1_drop,n_hidden2,activation=tf.nn.relu,name='hidden2')
    hidden2_drop = tf.layers.dropout(hidden2,dropout_rate,training=training)
    
    logits = tf.layers.dense(hidden2_drop,n_outputs,name='outputs')

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [127]:
tf.layers.dropout??

In [125]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [126]:
n_epochs = 20
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={training: True, X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Test accuracy: 0.9205
1 Test accuracy: 0.9362
2 Test accuracy: 0.9415
3 Test accuracy: 0.9508
4 Test accuracy: 0.9572
5 Test accuracy: 0.9557
6 Test accuracy: 0.9602
7 Test accuracy: 0.9602
8 Test accuracy: 0.9609
9 Test accuracy: 0.964
10 Test accuracy: 0.9633
11 Test accuracy: 0.9651
12 Test accuracy: 0.9667
13 Test accuracy: 0.9658
14 Test accuracy: 0.9658
15 Test accuracy: 0.9671
16 Test accuracy: 0.9685
17 Test accuracy: 0.9686
18 Test accuracy: 0.9672
19 Test accuracy: 0.9685


### 最大范数正则化

<div align=center><img src="./static/6.jpg"/></div>

#### 定义一个max_norm_regularizer()函数

`clip_by_norm()` 函数计算剪切后的权重。 然后我们创建一个赋值操作来将权值赋给权值变量：

In [ ]:
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None #  there is no regularization loss term
    return max_norm

然后你可以调用这个函数来得到一个最大范数调节器（与你想要的阈值）。 当你创建一个隐藏层时，你可以将这个正则化器传递给kernel_regularizer参数：

In [ ]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10
learning_rate = 0.01
momentum = 0.9
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [ ]:
max_norm_reg = max_norm_regularizer(threshold=1.0)
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [ ]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()
saver = tf.train.Saver()

请注意，最大范数正则化不需要在整体损失函数中添加正则化损失项，所以max_norm()函数返回None。 

但是，在每个训练步骤之后，仍需要运行clip_weights操作，因此您需要能够掌握它。 这就是为什么max_norm()函数将clip_weights节点添加到最大范数剪裁操作的集合中的原因。您需要获取这些裁剪操作并在每个训练步骤后运行它们：

In [ ]:
n_epochs = 20
batch_size = 50

clip_all_weights = tf.get_collection("max_norm")
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,     #  not shown in the book
                                            y: mnist.test.labels})    #  not shown
        print(epoch, "Test accuracy:", acc_test)                      #  not shown
    save_path = saver.save(sess, "./my_model_final.ckpt")             #  not shown